# Circular plate

Refer to the [lecture notes on classical plate theory](http://www.facweb.iitkgp.ac.in/~jeevanjyoti/teaching/advmechsolids/2020/notes/cpt.pdf) on the [Course Home Page of Advanced Mechanics of Solids](http://www.facweb.iitkgp.ac.in/~jeevanjyoti/teaching/advmechsolids/2020/)

For a plate with an arbitrarily shaped (smooth) periphery, the governing equation is:

\begin{gather*}
\nabla^4 w = q
\end{gather*}

and the boundary conditions are:

\begin{align*}
& \text{Either} \quad \frac{\partial M_{ns}}{\partial s} + Q_n = 0 \quad \text{or} \quad w \text{ is specified}; \\
& \text{Either} \quad M_n = 0 \quad \text{or} \quad \displaystyle \frac{\partial w}{\partial n} \text{ is specified.} 
\end{align*}

Here, $s$ and $n$ are the coordinate variables along the periphery and normal to the periphery, respectively. Futhermore, in terms of the moments and shear forces referred to the Cartesian coordinate system, we have:

\begin{align*}
M_n &= n_x^2 M_x + 2 n_x n_y M_{xy} + n_y^2 M_y, \\
M_{ns} &= n_x n_y (M_y - M_x) + (n_x^2 - n_y^2)M_{xy}, \\
Q_n &= n_x Q_x + n_y Q_y
\end{align*}

In the preceding relations, $n_x$ and $n_y$ are the components of the unit outward normal to the periphery (in the plane of the plate). 

We now apply this theory to the bending of circular plates. Then the coordinate variable $n$ from the general theory specializes to the radial coordinate variable $r$, whereas $s$ specializes to $r\theta$. 

The circular plates will be loaded and supported/fixed in ways such that axisymmetry is always maintained. In such cases, then, the deflection $w$ will solely be a function of the radial coordinate, $r$, and independent of $\theta$.  

## Simply-supported circular plate subjected to uniform transverse loading

We consider a circular plate of radius, $R$, subjected to a transverse loading of magnitude, $q$, and simply supported all along the periphery. Following Landau and Lifshitz, we will denote the ratio $\displaystyle \frac{q}{D}$ as $64\beta$; here $\displaystyle D = \frac{Eh^3}{12(1-\nu^2)}$ is the bending stiffness. The transverse deflection $w$ is solely a function of $r$. 

The governing equation is: $\displaystyle \nabla^4 w = 64 \beta$

and the boundary conditions are: $M_n \equiv M_r = 0$ and $w = 0$ at the periphery. 

In [ ]:
import sympy as sp

We import the `polarUtilities` module that we had created during Applied Elasticity. This will be useful in determining the expression for $M_r$.

In [ ]:
from polarUtilities import *

We note that 

\begin{align*}
M_x &= -D \left( \frac{\partial^2 w}{\partial x^2} + \nu \frac{\partial^2 w}{\partial y^2} \right), \\
M_y &= -D \left( \frac{\partial^2 w}{\partial y^2} + \nu \frac{\partial^2 w}{\partial x^2} \right), \\
M_{xy} &= -D(1-\nu)\frac{\partial^2 w}{\partial x \partial y}
\end{align*}

We utilize the definitions present in this module to express $M_x$, $M_y$, and $M_{xy}$ in terms of derivatives in the $r-\theta$ coordinate system. (Because of axisymmetry all derivatives with respect to $\theta$ will vanish).

In [58]:
w = sp.Function('w')(r)
D, nu = sp.symbols('D, nu',positive=True)


def Mx(w):
    return -D*(del_delx(del_delx(w)) + nu*del_dely(del_dely(w))).simplify()

def My(w):
    return -D*(del_dely(del_dely(w)) + nu*del_delx(del_delx(w))).simplify()

def Mxy(w):
    return -D*(1-nu)*del_delx(del_dely(w))

In the $r-\theta$ coordinate system for the circular plate, $n_x = \cos \theta$ and $n_y = \sin \theta$:

In [26]:
nx = sp.cos(theta)
ny = sp.sin(theta)

Then the expression for $M_n \equiv M_r$ can be determined:

In [38]:
def Mn(w):
    return (nx**2*Mx(w) + 2*nx*ny*Mxy(w) + ny**2*My(w)).simplify()

In [61]:
from IPython.display import Math

display(Math(r'M_r = {}'.format(sp.latex(Mn(w)))))

<IPython.core.display.Math object>

This matches with the expression given in Landau and Lifshitz. 

Next, we set up the governing equation:

In [31]:
beta = sp.symbols('beta')
eq = sp.Eq(polarbiharmonic(w), 64*beta)
eq

Eq(Derivative(w(r), (r, 4)) + 2*Derivative(w(r), (r, 3))/r - Derivative(w(r), (r, 2))/r**2 + Derivative(w(r), r)/r**3, 64*beta)

The solution is found as follows:

In [32]:
sp.dsolve(eq)

Eq(w(r), C1 + C2*r**2 + C3*r**2*log(r) + C4*log(r) + beta*r**4)

We rewrite the solution using $a$, $b$, $c$, and $d$ as the constants of integration:

In [62]:
a, b, c, d = sp.symbols('a, b, c, d')

w_soln = beta*r**4 + a*r**2*sp.log(r) + b*sp.log(r) + c*r**2 + d
display(Math(r'w = {}'.format(sp.latex(w_soln))))

<IPython.core.display.Math object>

The expression for $M_r$ then becomes:

In [63]:
Mr = Mn(w_soln)
display(Math(r'M_r = {}'.format(sp.latex(Mr))))

<IPython.core.display.Math object>

We note that for the deflection, $w(r)$ and the moment, $M_r$ to be finite as $r \to 0$, we must have $a = 0$ and $b=0$. Then the solution form becomes:

In [43]:
w_soln_interim = w_soln.subs([(a,0),(b,0)])
w_soln_interim

beta*r**4 + c*r**2 + d

We can now impose the two boundary conditions, $w = 0$ and $M_r = 0$ at $r = R$, i.e. along the periphery. From these two boundary conditions, we can find the values of $c$ and $d$:

In [64]:
R = sp.symbols('R',positive=True)

lhs1 = w_soln_interim.subs(r,R)
rhs1 = 0
bc1_eq = sp.Eq(lhs1,rhs1)

lhs2 = (Mn(w_soln_interim)).subs(r,R)
rhs2 = 0
bc2_eq = sp.Eq(lhs2,rhs2)

soln, = sp.linsolve([bc1_eq,bc2_eq],(c,d))
cvalue = soln[0]
dvalue = soln[1]

display(Math(r'c = {}'.format(sp.latex(cvalue))))
display(Math(r'd = {}'.format(sp.latex(dvalue))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The final solution is then:

In [65]:
w_soln_final = (w_soln_interim.subs([(c,cvalue),(d,dvalue)]))
display(Math(r'w_{{\rm soln}} = {}'.format(sp.latex(w_soln_final))))

<IPython.core.display.Math object>

This matches with the solution given in Landau and Lifshitz.